In [0]:
%tensorflow_version 2.x

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout, Conv1D, MaxPooling1D, GRU
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import os, re, json, functools

TensorFlow 2.x selected.


In [0]:
!pip install kaggle --upgrade

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


#Download the dataset

In [0]:
os.environ['KAGGLE_USERNAME'] = "adriancatalan"
os.environ['KAGGLE_KEY'] = "357e7685fa71bc4c22256523f32d7731"

In [0]:
#https://www.kaggle.com/extralime/math-lectures
!kaggle datasets download -d extralime/math-lectures

math-lectures.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
os.listdir()

['.config',
 'math-lectures.zip',
 'spam-text-message-classification.zip',
 'sample_data']

In [0]:
!unzip -o 'math-lectures.zip'

Archive:  math-lectures.zip
  inflating: raw_text.csv            


In [0]:
textmessage = pd.read_csv('raw_text.csv')
textmessage.head

<bound method NDFrame.head of                                                   text           label
0    The following content is\nprovided under a Cre...        Calculus
1    In this sequence of segments,\nwe review some ...     Probability
2    The following content is\nprovided under a Cre...              CS
3    The following\ncontent is provided under a Cre...      Algorithms
4    The following\ncontent is provided under a Cre...      Algorithms
..                                                 ...             ...
855  The following content is\nprovided under a Cre...   Math for Eng.
856  &gt;&gt; [MUSIC] &gt;&gt; DAVID J. MALAN: All ...       Diff. Eq.
857  The following content is\nprovided by MIT Open...  Linear Algebra
858  The following content is\nprovided under a Cre...              AI
859  The following content is\nprovided under a Cre...       Diff. Eq.

[860 rows x 2 columns]>

In [0]:
textmessage.columns

Index(['text', 'label'], dtype='object')

In [0]:
textmessage.text

0      The following content is\nprovided under a Cre...
1      In this sequence of segments,\nwe review some ...
2      The following content is\nprovided under a Cre...
3      The following\ncontent is provided under a Cre...
4      The following\ncontent is provided under a Cre...
                             ...                        
855    The following content is\nprovided under a Cre...
856    &gt;&gt; [MUSIC] &gt;&gt; DAVID J. MALAN: All ...
857    The following content is\nprovided by MIT Open...
858    The following content is\nprovided under a Cre...
859    The following content is\nprovided under a Cre...
Name: text, Length: 860, dtype: object

In [0]:
textmessage.label

0            Calculus
1         Probability
2                  CS
3          Algorithms
4          Algorithms
            ...      
855     Math for Eng.
856         Diff. Eq.
857    Linear Algebra
858                AI
859         Diff. Eq.
Name: label, Length: 860, dtype: object

In [0]:

textmessage['Calculus'] = textmessage['label'].apply(lambda x: 1 if x == "Calculus" else 0)
textmessage.drop(columns=['label'])

,text,Algorithms,Calculus
0,The following content is\nprovided under a Cre...,0,1
1,"In this sequence of segments,\nwe review some ...",0,0
2,The following content is\nprovided under a Cre...,0,0
3,The following\ncontent is provided under a Cre...,1,0
4,The following\ncontent is provided under a Cre...,1,0
...,...,...,...
855,The following content is\nprovided under a Cre...,0,0
856,&gt;&gt; [MUSIC] &gt;&gt; DAVID J. MALAN: All ...,0,0
857,The following content is\nprovided by MIT Open...,0,0
858,The following content is\nprovided under a Cre...,0,0


In [0]:
text = textmessage.text
label = textmessage.label
textmessage_train , label_test, label_train, textmessage_test = train_test_split(text,label,test_size=0.3)

#Build the model

In [0]:
MAX_WORDS = 860
MAX_LEN = 100
BATCH_SIZE = 688
EPOCHS = 5

In [0]:
tokenizer = Tokenizer(num_words=860)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
matrix_sequences = pad_sequences(sequences,maxlen=100)

In [0]:
model = Sequential([
    Embedding(860, 32, mask_zero=True),
    Bidirectional(LSTM(64)),
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 32)          27520     
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128)               49664     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 77,313
Trainable params: 77,313
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#Training the model

In [0]:
model.fit(matrix_sequences,label,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.2)

Train on 688 samples, validate on 172 samples
Epoch 1/5
688/688 [==============================] - 4s 5ms/sample


UnimplementedError: ignored

In [0]:
test_sequences = tokenizer.texts_to_sequences(text)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=MAX_LEN)

In [0]:
accr = model.evaluate(test_sequences_matrix,label)
print('Loss: {:0.3f} y  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

 32/860 [>.............................] - ETA: 0s

UnimplementedError: ignored